In [16]:
# AI Internship
# Sushant_Kawade

In [17]:
# Necessary Imports
import cv2
import numpy as np

In [18]:
# Define the range of colors for the balls
lower_yellow = (20, 100, 100)
upper_yellow = (30, 255, 255)

lower_cyan = (78, 43, 46)
upper_cyan = (98, 255, 255)

lower_peach = (5,100, 100)
upper_peach = (20, 255, 255)

l_bond = (155,25,0)
u_bond = (179,255,255)

In [19]:
# Create a VideoCapture object to read the video
cap = cv2.VideoCapture("AI_Assignment_Video.mp4")

In [20]:
# Get Frame Height & Width
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [21]:
# LOOP

while True:
    
    # Read a frame from the video
    ret, frame = cap.read()
    
    # Divide the frame into four equal parts
    top_left = frame[0:frame_height//2, 0:frame_width//2]
    top_right = frame[0:frame_height//2, frame_width//2:frame_width]
    bottom_left = frame[frame_height//2:frame_height, 0:frame_width//2]
    bottom_right = frame[frame_height//2:frame_height, frame_width//2:frame_width]
    

    # Draw rectangles around each quadrant
    #quad1
    cv2.rectangle(frame, (0, 0), (frame_width//2, frame_height//2), (0, 255, 0), 2)
    #quad2
    cv2.rectangle(frame, (frame_width//2, 0), (frame_width, frame_height//2), (0, 255, 0), 2)
    #quad3
    cv2.rectangle(frame, (frame_width//2, frame_height//2), (frame_width, frame_height), (0, 255, 0), 2)
    #quad4
    cv2.rectangle(frame, (0, frame_height//2), (frame_width//2, frame_height), (0, 255, 0), 2)
    
    
    # Check if the video has ended
    if not ret:
        break

        
    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    
    # Threshold the frame to select the colors (MASKING)
    mask1 = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask2 = cv2.inRange(hsv, lower_cyan, upper_cyan)
    mask3 = cv2.inRange(hsv, lower_peach, upper_peach)
    mask_bond = cv2.inRange(hsv, l_bond, u_bond)
    mask = mask1 + mask2 + mask3 
    
    
    # Use a morphological operation to remove noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    

    # Find the contours of the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
 
    # Draw the contours on the frame
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
        
    # finding the mask with maximum area    
    max_area = 0
    max_cnt = None
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > max_area:
            max_area = area
            max_cnt = cnt
    
    
    # Finding center of the masked object (ball)
    if max_cnt is not None:
        M = cv2.moments(max_cnt)
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        
    # Setting Center point to get HSV range (HUE value)
    pixel_center = hsv[cy, cx]
    hue_value = pixel_center[0]

    color = "Undefined"
    if hue_value < 20:
        color = "PEACH"
    elif hue_value < 33:
        color = "YELLOW"
    elif hue_value < 98:
        color = "CYAN"
    
    pixel_center_bgr = frame[cy, cx]
    b, g, r = int(pixel_center_bgr[0]), int(pixel_center_bgr[1]), int(pixel_center_bgr[2])
    cv2.rectangle(frame, (280, 10), (420, 60), (255, 255, 255), -1)
    cv2.putText(frame, color, (290,40), 0, 1, (b, g, r), 2)
    cv2.circle(frame, (cx, cy), 5, (25, 25, 25), 3)

    
    # Find the bounding box of the object
    x, y, w, h = cv2.boundingRect(max_cnt[0])

    
    # Determine in which quadrant the object is located
    if x < frame.shape[1]/2 and y < frame.shape[0]/2:
        cv2.putText(frame, 'The object is in the top-left (1st) quadrant.', (x-30, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

    elif x + w >= frame.shape[1]/2 and y < frame.shape[0]/2:
        cv2.putText(frame, 'The object is in the top-right (2nd) quadrant.', (x-30, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

    elif x < frame.shape[1]/2 and y + h >= frame.shape[0]/2:
        cv2.putText(frame, 'The object is in the bottom-left (4th) quadrant.', (x-30, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

    else:
        cv2.putText(frame, 'The object is in the bottom-right (3rd) quadrant.', (x-30, y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

        
    # Display the frame
    cv2.imshow("Detected Balls", frame)

    
    # Check if the user pressed the 'q' key to QUIT
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
# Release the VideoCapture object
cap.release()
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable